In [1]:
# General imports
import numpy as np
import pandas as pd
import os, sys, gc, time, warnings, pickle, psutil, random

# custom imports
from multiprocessing import Pool        # Multiprocess Runs

warnings.filterwarnings('ignore')

In [2]:
# :seed to make all processes deterministic     # type: int
def seed_everything(seed=0):
    random.seed(seed)
    np.random.seed(seed)

    
## Multiprocess Runs
def df_parallelize_run(func, t_split):
    num_cores = np.min([N_CORES,len(t_split)])
    pool = Pool(num_cores)
    df = pd.concat(pool.map(func, t_split), axis=1)
    pool.close()
    pool.join()
    return df

In [3]:
# Read data
def get_data_by_store(store):
    
    # Read and contact basic feature
    df = pd.concat([pd.read_pickle(BASE),
                    pd.read_pickle(PRICE).iloc[:,2:],
                    pd.read_pickle(CALENDAR).iloc[:,2:]],
                    axis=1)
    
    # Leave only relevant store
    df = df[df['store_id']==store]

    # With memory limits we have to read 
    # lags and mean encoding features
    # separately and drop items that we don't need.
    # As our Features Grids are aligned 
    # we can use index to keep only necessary rows
    # Alignment is good for us as concat uses less memory than merge.
    df2 = pd.read_pickle(MEAN_ENC)[mean_features]
    df2 = df2[df2.index.isin(df.index)]
    
    df3 = pd.read_pickle(LAGS).iloc[:,3:]
    df3 = df3[df3.index.isin(df.index)]
    
    df = pd.concat([df, df2], axis=1)
    del df2 # to not reach memory limit 
    
    df = pd.concat([df, df3], axis=1)
    del df3 # to not reach memory limit 
    
    # Create features list
    features = [col for col in list(df) if col not in remove_features]
    df = df[['id','d',TARGET]+features]
    
    # Skipping first n rows
    df = df[df['d']>=START_TRAIN].reset_index(drop=True)
    
    return df, features

# Recombine Test set after training
def get_base_test():
    base_test = pd.DataFrame()

    for store_id in STORES_IDS:
        temp_df = pd.read_pickle('test_'+store_id+'.pkl')
        temp_df['store_id'] = store_id
        base_test = pd.concat([base_test, temp_df]).reset_index(drop=True)
    
    return base_test


########################### Helper to make dynamic rolling lags
#################################################################################
def make_lag(LAG_DAY):
    lag_df = base_test[['id','d',TARGET]]
    col_name = 'sales_lag_'+str(LAG_DAY)
    lag_df[col_name] = lag_df.groupby(['id'])[TARGET].transform(lambda x: x.shift(LAG_DAY)).astype(np.float16)
    return lag_df[[col_name]]


def make_lag_roll(LAG_DAY):
    shift_day = LAG_DAY[0]
    roll_wind = LAG_DAY[1]
    lag_df = base_test[['id','d',TARGET]]
    col_name = 'rolling_mean_tmp_'+str(shift_day)+'_'+str(roll_wind)
    lag_df[col_name] = lag_df.groupby(['id'])[TARGET].transform(lambda x: x.shift(shift_day).rolling(roll_wind).mean())
    return lag_df[[col_name]]

In [4]:
import lightgbm as lgb
lgb_params = {
                    'boosting_type': 'gbdt',
                    'objective': 'tweedie',
                    'tweedie_variance_power': 1.1,
                    'metric': 'rmse',
                    'subsample': 0.5,
                    'subsample_freq': 1,
                    'learning_rate': 0.03,
                    'num_leaves': 2**11-1,
                    'min_data_in_leaf': 2**12-1,
                    'feature_fraction': 0.5,
                    'max_bin': 100,
                    'n_estimators': 1400,
                    'boost_from_average': False,
                    'verbose': -1,
                } 

# Let's look closer on params

## 'boosting_type': 'gbdt'
# we have 'goss' option for faster training
# but it normally leads to underfit.
# Also there is good 'dart' mode
# but it takes forever to train
# and model performance depends 
# a lot on random factor 
# https://www.kaggle.com/c/home-credit-default-risk/discussion/60921

## 'objective': 'tweedie'
# Tweedie Gradient Boosting for Extremely
# Unbalanced Zero-inflated Data
# https://arxiv.org/pdf/1811.10192.pdf
# and many more articles about tweediie
#
# Strange (for me) but Tweedie is close in results
# to my own ugly loss.
# My advice here - make OWN LOSS function
# https://www.kaggle.com/c/m5-forecasting-accuracy/discussion/140564
# https://www.kaggle.com/c/m5-forecasting-accuracy/discussion/143070
# I think many of you already using it (after poisson kernel appeared) 
# (kagglers are very good with "params" testing and tuning).
# Try to figure out why Tweedie works.
# probably it will show you new features options
# or data transformation (Target transformation?).

## 'tweedie_variance_power': 1.1
# default = 1.5
# set this closer to 2 to shift towards a Gamma distribution
# set this closer to 1 to shift towards a Poisson distribution
# my CV shows 1.1 is optimal 
# but you can make your own choice

## 'metric': 'rmse'
# Doesn't mean anything to us
# as competition metric is different
# and we don't use early stoppings here.
# So rmse serves just for general 
# model performance overview.
# Also we use "fake" validation set
# (as it makes part of the training set)
# so even general rmse score doesn't mean anything))
# https://www.kaggle.com/c/m5-forecasting-accuracy/discussion/133834

## 'subsample': 0.5
# Serves to fight with overfit
# this will randomly select part of data without resampling
# Chosen by CV (my CV can be wrong!)
# Next kernel will be about CV

##'subsample_freq': 1
# frequency for bagging
# default value - seems ok

## 'learning_rate': 0.03
# Chosen by CV
# Smaller - longer training
# but there is an option to stop 
# in "local minimum"
# Bigger - faster training
# but there is a chance to
# not find "global minimum" minimum

## 'num_leaves': 2**11-1
## 'min_data_in_leaf': 2**12-1
# Force model to use more features
# We need it to reduce "recursive"
# error impact.
# Also it leads to overfit
# that's why we use small 
# 'max_bin': 100

## l1, l2 regularizations
# https://towardsdatascience.com/l1-and-l2-regularization-methods-ce25e7fc831c
# Good tiny explanation
# l2 can work with bigger num_leaves
# but my CV doesn't show boost
                    
## 'n_estimators': 1400
# CV shows that there should be
# different values for each state/store.
# Current value was chosen 
# for general purpose.
# As we don't use any early stopings
# careful to not overfit Public LB.

##'feature_fraction': 0.5
# LightGBM will randomly select 
# part of features on each iteration (tree).
# We have maaaany features
# and many of them are "duplicates"
# and many just "noise"
# good values here - 0.5-0.7 (by CV)

## 'boost_from_average': False
# There is some "problem"
# to code boost_from_average for 
# custom loss
# 'True' makes training faster
# BUT carefull use it
# https://github.com/microsoft/LightGBM/issues/1514

In [5]:
VER = 1                          # Our model version
SEED = 42                        # We want all things
seed_everything(SEED)            # to be as deterministic 
lgb_params['seed'] = SEED        # as possible
N_CORES = psutil.cpu_count()     # Available CPU cores


#LIMITS and const
TARGET      = 'sales'            # Our target
START_TRAIN = 0                  # We can skip some rows (Nans/faster training)
END_TRAIN   = 1913               # End day of our train set
P_HORIZON   = 28                 # Prediction horizon
USE_AUX     = True               # Use or not pretrained models

#FEATURES to remove
## These features lead to overfit
## or values not present in test set
remove_features = ['id','state_id','store_id',
                   'date','wm_yr_wk','d',TARGET]
mean_features   = ['enc_cat_id_mean','enc_cat_id_std',
                   'enc_dept_id_mean','enc_dept_id_std',
                   'enc_item_id_mean','enc_item_id_std'] 

#PATHS for Features
BASE     = 'grid_part_1.pkl'
PRICE    = 'grid_part_2.pkl'
CALENDAR = 'grid_part_3.pkl'
LAGS     = 'lags_df_28.pkl'
MEAN_ENC = 'mean_encoding_df.pkl'


# AUX(pretrained) Models paths

#STORES ids
STORES_IDS = pd.read_csv('sales_train_validation.csv')['store_id']
STORES_IDS = list(STORES_IDS.unique())


#SPLITS for lags creation
SHIFT_DAY  = 28
N_LAGS     = 15
LAGS_SPLIT = [col for col in range(SHIFT_DAY,SHIFT_DAY+N_LAGS)]
ROLS_SPLIT = []
for i in [1,7,14]:
    for j in [7,14,30,60]:
        ROLS_SPLIT.append([i,j])

In [8]:
for store_id in STORES_IDS:
    print('Train', store_id)
    
    # Get grid for current store
    grid_df, features_columns = get_data_by_store(store_id)
    
    # Masks for 
    # Train (All data less than 1913)
    # "Validation" (Last 28 days - not real validation set)
    # Test (All data greater than 1913 day, 
    #       with some gap for recursive features)
    train_mask = grid_df['d']<=END_TRAIN
    valid_mask = train_mask&(grid_df['d']>(END_TRAIN-P_HORIZON))
    preds_mask = grid_df['d']>(END_TRAIN-100)
    
    # Apply masks and save lgb dataset as bin
    # to reduce memory spikes during dtype convertations
    # https://github.com/Microsoft/LightGBM/issues/1032
    # "To avoid any conversions, you should always use np.float32"
    # or save to bin before start training
    # https://www.kaggle.com/c/talkingdata-adtracking-fraud-detection/discussion/53773
    train_data = lgb.Dataset(grid_df[train_mask][features_columns], 
                       label=grid_df[train_mask][TARGET])
    train_data.save_binary('train_data.bin')
    train_data = lgb.Dataset('train_data.bin')
    
    valid_data = lgb.Dataset(grid_df[valid_mask][features_columns], 
                       label=grid_df[valid_mask][TARGET])
    
    # Saving part of the dataset for later predictions
    # Removing features that we need to calculate recursively 
    grid_df = grid_df[preds_mask].reset_index(drop=True)
    keep_cols = [col for col in list(grid_df) if '_tmp_' not in col]
    grid_df = grid_df[keep_cols]
    print(grid_df.head())
    print(grid_df.tail())
    print(grid_df.info())
    grid_df.to_pickle('test_'+store_id+'.pkl')
    del grid_df
    
    # Launch seeder again to make lgb training 100% deterministic
    # with each "code line" np.random "evolves" 
    # so we need (may want) to "reset" it
    seed_everything(SEED)
    estimator = lgb.train(lgb_params,
                          train_data,
                          valid_sets = [valid_data],
                          verbose_eval = 100,
                          )
    
    # Save model - it's not real '.bin' but a pickle file
    # estimator = lgb.Booster(model_file='model.txt')
    # can only predict with the best iteration (or the saving iteration)
    # pickle.dump gives us more flexibility
    # like estimator.predict(TEST, num_iteration=100)
    # num_iteration - number of iteration want to predict with, 
    # NULL or <= 0 means use best iteration
    model_name = 'lgb_model_'+store_id+'_v'+str(VER)+'.bin'
    pickle.dump(estimator, open(model_name, 'wb'))

    # Remove temporary files and objects 
    # to free some hdd space and ram memory
    !rm train_data.bin
    del train_data, valid_data, estimator
    gc.collect()
    
    # "Keep" models features for predictions
    MODEL_FEATURES = features_columns

Train CA_1
                              id     d  sales        item_id    dept_id  \
0  HOBBIES_1_001_CA_1_validation  1814    0.0  HOBBIES_1_001  HOBBIES_1   
1  HOBBIES_1_002_CA_1_validation  1814    0.0  HOBBIES_1_002  HOBBIES_1   
2  HOBBIES_1_003_CA_1_validation  1814    0.0  HOBBIES_1_003  HOBBIES_1   
3  HOBBIES_1_004_CA_1_validation  1814    3.0  HOBBIES_1_004  HOBBIES_1   
4  HOBBIES_1_005_CA_1_validation  1814    0.0  HOBBIES_1_005  HOBBIES_1   

    cat_id  release  sell_price  price_max  price_min  ...  rolling_mean_7  \
0  HOBBIES      224    8.257812   9.578125   8.257812  ...        0.714355   
1  HOBBIES       20    3.970703   3.970703   3.970703  ...        1.428711   
2  HOBBIES      300    2.970703   2.970703   2.970703  ...        1.857422   
3  HOBBIES        5    4.640625   4.640625   4.339844  ...        1.713867   
4  HOBBIES       16    2.880859   3.080078   2.480469  ...        2.572266   

   rolling_std_7  rolling_mean_14  rolling_std_14  rolling_mean_30  \

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 390216 entries, 0 to 390215
Data columns (total 63 columns):
id                  390216 non-null category
d                   390216 non-null int16
sales               304844 non-null float64
item_id             390216 non-null category
dept_id             390216 non-null category
cat_id              390216 non-null category
release             390216 non-null int16
sell_price          390216 non-null float16
price_max           390216 non-null float16
price_min           390216 non-null float16
price_std           390216 non-null float16
price_mean          390216 non-null float16
price_norm          390216 non-null float16
price_nunique       390216 non-null float16
item_nunique        390216 non-null int16
price_momentum      390195 non-null float16
price_momentum_m    390216 non-null float16
price_momentum_y    390216 non-null float16
event_name_1        39631 non-null category
event_type_1        39631 non-null category
event_name_

[100]	valid_0's rmse: 2.5055
[200]	valid_0's rmse: 2.46208
[300]	valid_0's rmse: 2.43993
[400]	valid_0's rmse: 2.42454
[500]	valid_0's rmse: 2.41549
[600]	valid_0's rmse: 2.40771
[700]	valid_0's rmse: 2.40195
[800]	valid_0's rmse: 2.39547
[900]	valid_0's rmse: 2.38985
[1000]	valid_0's rmse: 2.38472
[1100]	valid_0's rmse: 2.38037
[1200]	valid_0's rmse: 2.37576
[1300]	valid_0's rmse: 2.37126
[1400]	valid_0's rmse: 2.36679
Train CA_4
                              id     d  sales        item_id    dept_id  \
0  HOBBIES_1_001_CA_4_validation  1814    1.0  HOBBIES_1_001  HOBBIES_1   
1  HOBBIES_1_002_CA_4_validation  1814    0.0  HOBBIES_1_002  HOBBIES_1   
2  HOBBIES_1_003_CA_4_validation  1814    0.0  HOBBIES_1_003  HOBBIES_1   
3  HOBBIES_1_004_CA_4_validation  1814    0.0  HOBBIES_1_004  HOBBIES_1   
4  HOBBIES_1_005_CA_4_validation  1814    0.0  HOBBIES_1_005  HOBBIES_1   

    cat_id  release  sell_price  price_max  price_min  ...  rolling_mean_7  \
0  HOBBIES      224    8.257812   9.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 390272 entries, 0 to 390271
Data columns (total 63 columns):
id                  390272 non-null category
d                   390272 non-null int16
sales               304900 non-null float64
item_id             390272 non-null category
dept_id             390272 non-null category
cat_id              390272 non-null category
release             390272 non-null int16
sell_price          390272 non-null float16
price_max           390272 non-null float16
price_min           390272 non-null float16
price_std           390272 non-null float16
price_mean          390272 non-null float16
price_norm          390272 non-null float16
price_nunique       390272 non-null float16
item_nunique        390272 non-null int16
price_momentum      390272 non-null float16
price_momentum_m    390272 non-null float16
price_momentum_y    390272 non-null float16
event_name_1        39637 non-null category
event_type_1        39637 non-null category
event_name_

[100]	valid_0's rmse: 1.71833
[200]	valid_0's rmse: 1.69794
[300]	valid_0's rmse: 1.68458
[400]	valid_0's rmse: 1.67864
[500]	valid_0's rmse: 1.67315
[600]	valid_0's rmse: 1.66851
[700]	valid_0's rmse: 1.66399
[800]	valid_0's rmse: 1.66003
[900]	valid_0's rmse: 1.65663
[1000]	valid_0's rmse: 1.65273
[1100]	valid_0's rmse: 1.64889
[1200]	valid_0's rmse: 1.6459
[1300]	valid_0's rmse: 1.64224
[1400]	valid_0's rmse: 1.63887
Train TX_3
                              id     d  sales        item_id    dept_id  \
0  HOBBIES_1_001_TX_3_validation  1814    0.0  HOBBIES_1_001  HOBBIES_1   
1  HOBBIES_1_002_TX_3_validation  1814    0.0  HOBBIES_1_002  HOBBIES_1   
2  HOBBIES_1_003_TX_3_validation  1814    0.0  HOBBIES_1_003  HOBBIES_1   
3  HOBBIES_1_004_TX_3_validation  1814    1.0  HOBBIES_1_004  HOBBIES_1   
4  HOBBIES_1_005_TX_3_validation  1814    1.0  HOBBIES_1_005  HOBBIES_1   

    cat_id  release  sell_price  price_max  price_min  ...  rolling_mean_7  \
0  HOBBIES      226    8.257812   8.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 390244 entries, 0 to 390243
Data columns (total 63 columns):
id                  390244 non-null category
d                   390244 non-null int16
sales               304872 non-null float64
item_id             390244 non-null category
dept_id             390244 non-null category
cat_id              390244 non-null category
release             390244 non-null int16
sell_price          390244 non-null float16
price_max           390244 non-null float16
price_min           390244 non-null float16
price_std           390244 non-null float16
price_mean          390244 non-null float16
price_norm          390244 non-null float16
price_nunique       390244 non-null float16
item_nunique        390244 non-null int16
price_momentum      390237 non-null float16
price_momentum_m    390244 non-null float16
price_momentum_y    390244 non-null float16
event_name_1        39634 non-null category
event_type_1        39634 non-null category
event_name_

[100]	valid_0's rmse: 2.69105
[200]	valid_0's rmse: 2.58171
[300]	valid_0's rmse: 2.54574
[400]	valid_0's rmse: 2.52407
[500]	valid_0's rmse: 2.50655
[600]	valid_0's rmse: 2.49078
[700]	valid_0's rmse: 2.47712
[800]	valid_0's rmse: 2.46442
[900]	valid_0's rmse: 2.45332
[1000]	valid_0's rmse: 2.44306
[1100]	valid_0's rmse: 2.43294
[1200]	valid_0's rmse: 2.42588
[1300]	valid_0's rmse: 2.41622
[1400]	valid_0's rmse: 2.40552
Train WI_3
                              id     d  sales        item_id    dept_id  \
0  HOBBIES_1_001_WI_3_validation  1814    2.0  HOBBIES_1_001  HOBBIES_1   
1  HOBBIES_1_002_WI_3_validation  1814    0.0  HOBBIES_1_002  HOBBIES_1   
2  HOBBIES_1_003_WI_3_validation  1814    1.0  HOBBIES_1_003  HOBBIES_1   
3  HOBBIES_1_004_WI_3_validation  1814    3.0  HOBBIES_1_004  HOBBIES_1   
4  HOBBIES_1_005_WI_3_validation  1814    3.0  HOBBIES_1_005  HOBBIES_1   

    cat_id  release  sell_price  price_max  price_min  ...  rolling_mean_7  \
0  HOBBIES      227    8.257812   8

In [9]:
# Create Dummy DataFrame to store predictions
all_preds = pd.DataFrame()

# Join back the Test dataset with 
# a small part of the training data 
# to make recursive features
base_test = get_base_test()

# Timer to measure predictions time 
main_time = time.time()

# Loop over each prediction day
# As rolling lags are the most timeconsuming
# we will calculate it for whole day
for PREDICT_DAY in range(1,29):    
    print('Predict | Day:', PREDICT_DAY)
    start_time = time.time()

    # Make temporary grid to calculate rolling lags
    grid_df = base_test.copy()
    grid_df = pd.concat([grid_df, df_parallelize_run(make_lag_roll, ROLS_SPLIT)], axis=1)
        
    for store_id in STORES_IDS:
        
        # Read all our models and make predictions
        # for each day/store pairs
        model_path = 'lgb_model_'+store_id+'_v'+str(VER)+'.bin' 
        
        estimator = pickle.load(open(model_path, 'rb'))
        
        day_mask = base_test['d']==(END_TRAIN+PREDICT_DAY)
        store_mask = base_test['store_id']==store_id
        
        mask = (day_mask)&(store_mask)
        base_test[TARGET][mask] = estimator.predict(grid_df[mask][MODEL_FEATURES])
    
    # Make good column naming and add 
    # to all_preds DataFrame
    temp_df = base_test[day_mask][['id',TARGET]]
    temp_df.columns = ['id','F'+str(PREDICT_DAY)]
    if 'id' in list(all_preds):
        all_preds = all_preds.merge(temp_df, on=['id'], how='left')
    else:
        all_preds = temp_df.copy()
        
    print('#'*10, ' %0.2f min round |' % ((time.time() - start_time) / 60),
                  ' %0.2f min total |' % ((time.time() - main_time) / 60),
                  ' %0.2f day sales |' % (temp_df['F'+str(PREDICT_DAY)].sum()))
    del temp_df
    
all_preds = all_preds.reset_index(drop=True)
all_preds.head()
all_preds.tail()

Predict | Day: 1
##########  0.77 min round |  0.77 min total |  37308.80 day sales |
Predict | Day: 2
##########  0.76 min round |  1.53 min total |  35335.42 day sales |
Predict | Day: 3
##########  0.74 min round |  2.27 min total |  34783.90 day sales |
Predict | Day: 4
##########  0.74 min round |  3.01 min total |  35285.85 day sales |
Predict | Day: 5
##########  0.75 min round |  3.76 min total |  41724.47 day sales |
Predict | Day: 6
##########  0.74 min round |  4.50 min total |  50966.54 day sales |
Predict | Day: 7
##########  0.74 min round |  5.24 min total |  53580.33 day sales |
Predict | Day: 8
##########  0.76 min round |  6.00 min total |  44119.60 day sales |
Predict | Day: 9
##########  0.74 min round |  6.74 min total |  44431.43 day sales |
Predict | Day: 10
##########  0.75 min round |  7.49 min total |  38864.02 day sales |
Predict | Day: 11
##########  0.75 min round |  8.24 min total |  40720.81 day sales |
Predict | Day: 12
##########  0.75 min round |  8.99

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
30485,FOODS_3_823_WI_3_validation,0.373569,0.341489,0.343651,0.330851,0.378356,0.431511,0.423120,0.463942,0.487156,...,0.406368,0.521859,0.573814,0.373389,0.333836,0.301846,0.283167,0.321126,0.396755,0.410307
30486,FOODS_3_824_WI_3_validation,0.304983,0.266182,0.243182,0.240461,0.294768,0.369417,0.331245,0.434473,0.442417,...,0.339828,0.490687,0.559283,0.363130,0.275342,0.245768,0.248248,0.271593,0.345999,0.361340
30487,FOODS_3_825_WI_3_validation,0.674252,0.539741,0.483850,0.463280,0.636023,0.735115,0.909188,1.166860,1.090199,...,1.004481,1.541840,1.688544,1.077601,0.795546,0.742795,0.649474,0.735476,0.864346,0.916464
30488,FOODS_3_826_WI_3_validation,0.944403,0.935210,0.808110,0.792187,0.949410,1.160681,1.166340,1.236102,1.229442,...,0.986986,1.370988,1.437660,1.000928,0.884545,0.845057,0.827194,0.980560,1.152899,1.231655
30489,FOODS_3_827_WI_3_validation,0.239573,1.224351,1.224150,2.056664,2.718922,2.748200,2.405680,2.231635,2.395505,...,1.971739,2.425102,2.260457,1.649259,1.674562,1.553669,1.488599,1.722757,2.069605,1.927087


In [11]:
# Reading competition sample submission and
# merging our predictions
# As we have predictions only for "_validation" data
# we need to do fillna() for "_evaluation" items
submission = pd.read_csv('sample_submission.csv')[['id']]
submission = submission.merge(all_preds, on=['id'], how='left').fillna(0)
submission.to_csv('submission_v'+str(VER)+'.csv', index=False) #0.47388

In [ ]:
# next step:

# Improvement should come from:
# bayesian optimization: store_id
# Loss function: rmse-wrmse
# Stable CV: by week-year
# Good features reduction strategy
# Predictions stabilization with NN
# Trend prediction

In [ ]:
#bayesian optimization
